<h1  style="text-align: center"  class="list-group-item list-group-item-warning"> <span style="color:#2E6AA9">Loan Model Building</span>  </h1>

In [1]:
#Basic Librariesused for Data Manupalution
import pandas as pd
import numpy as np
# Libraries for models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier
# Libraries performing Operation on Models
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report,f1_score

In [2]:
data=pd.read_csv('Processed_data_without_ouliters_02_07_22.csv',index_col='SK_ID_CURR')

In [3]:
X = data.drop(columns=['TARGET'])
Y = data.TARGET

In [4]:
# Cheacking the Class distribution of dependent variable
print(Y.value_counts()[1],Y.value_counts()[0])

24755 281458


#### Spliting the data set to test and train and then Over Sampling to balance the Target Class

In [5]:
# Spliting Data
xtrain1,xtest,ytrain1,ytest = train_test_split(X,Y,test_size=.25,shuffle=True,stratify=Y,random_state=23)

In [6]:
xtrain1.shape,xtest.shape,ytrain1.shape,ytest.shape

((229659, 47), (76554, 47), (229659,), (76554,))

from imblearn.over_sampling import SMOTE

smote = SMOTE()

# fit predictor and target variable
xtrain,ytrain = smote.fit_resample(xtrain, ytrain)
#xtest, ytest= smote.fit_resample(xtest, ytest)
print('Resample dataset shape',ytrain.value_counts())
print('Resample dataset shape',ytest.value_counts())

In [7]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)

xtrain, ytrain= ros.fit_resample(xtrain1, ytrain1)

print('original dataset shape:',ytrain1.value_counts())
print('Resample dataset shape',ytrain.value_counts())

original dataset shape: 0    211093
1     18566
Name: TARGET, dtype: int64
Resample dataset shape 0    211093
1    211093
Name: TARGET, dtype: int64


In [8]:
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((422186, 47), (76554, 47), (422186,), (76554,))

In [10]:
#Scaling the Data
scaler=StandardScaler()
xtrain=scaler.fit_transform(xtrain)
xtest=scaler.transform(xtest)

In [11]:
#Creating the LogisticRegression object
lg_model = LogisticRegression(random_state=23)

In [12]:
lg_model.fit(xtrain,ytrain)

LogisticRegression(random_state=23)

In [13]:
#predicting with seen Data
p1_ytrain=lg_model.predict(xtrain)

In [14]:
accuracy_score(ytrain,p1_ytrain),f1_score(ytrain,p1_ytrain)

(0.6172753241462294, 0.6196724938624965)

In [15]:
#predicting with Unseen Data
p1_ytest=lg_model.predict(xtest)

In [16]:
accuracy_score(ytest,p1_ytest),f1_score(ytest,p1_ytest)

(0.6131880763905218, 0.20312163616792248)

In [17]:
print(classification_report(ytrain,p1_ytrain),classification_report(ytest,p1_ytest))

              precision    recall  f1-score   support

           0       0.62      0.61      0.61    211093
           1       0.62      0.62      0.62    211093

    accuracy                           0.62    422186
   macro avg       0.62      0.62      0.62    422186
weighted avg       0.62      0.62      0.62    422186
               precision    recall  f1-score   support

           0       0.95      0.61      0.74     70365
           1       0.12      0.61      0.20      6189

    accuracy                           0.61     76554
   macro avg       0.53      0.61      0.47     76554
weighted avg       0.88      0.61      0.70     76554



In [18]:
confusion_matrix(ytest,p1_ytest)

array([[43168, 27197],
       [ 2415,  3774]], dtype=int64)

In [21]:
#lg_model.f

In [22]:
#creting RandomForestClassifier Object
rfc_model=RandomForestClassifier(random_state=23)

In [23]:
rfc_model.fit(xtrain,ytrain)

RandomForestClassifier(random_state=23)

In [24]:
#predicting with seen Data
p2_ytrain=rfc_model.predict(xtrain)

In [25]:
accuracy_score(ytrain,p2_ytrain)

1.0

In [26]:
#predicting with Unseen Data
p2_ytest=rfc_model.predict(xtest)

In [27]:
accuracy_score(ytest,p2_ytest)

0.9189461034041331

In [30]:
print(classification_report(ytrain,p2_ytrain),classification_report(ytest,p2_ytest))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    211093
           1       1.00      1.00      1.00    211093

    accuracy                           1.00    422186
   macro avg       1.00      1.00      1.00    422186
weighted avg       1.00      1.00      1.00    422186
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     70365
           1       0.29      0.00      0.00      6189

    accuracy                           0.92     76554
   macro avg       0.60      0.50      0.48     76554
weighted avg       0.87      0.92      0.88     76554



#### Hyper Parameter tuning

In [54]:
rf_params = {
    #'n_estimators': [10, 20, 30],
    #'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50,80],
    'min_samples_leaf': [1,2,4,8,10],
    "bootstrap":[True,False],
    #"criterion":['gini','entropy']
}

In [55]:
clf = rfc_model
rfc_grid = GridSearchCV(clf, rf_params, cv=3, scoring='f1',n_jobs=-1)
rfc_grid.fit(xtrain,ytrain)
print(rfc_grid.best_params_)
print("Accuracy:"+ str(rfc_grid.best_score_))

{'bootstrap': False, 'max_depth': 50, 'min_samples_leaf': 1}
Accuracy:0.9997702521010167


In [57]:
p_ytrain=rfc_grid.predict(xtrain)

In [58]:
p_ytest=rfc_grid.predict(xtest)

In [59]:
print(classification_report(ytrain,p_ytrain),classification_report(ytest,p_ytest))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    211093
           1       1.00      1.00      1.00    211093

    accuracy                           1.00    422186
   macro avg       1.00      1.00      1.00    422186
weighted avg       1.00      1.00      1.00    422186
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     70365
           1       0.21      0.00      0.00      6189

    accuracy                           0.92     76554
   macro avg       0.57      0.50      0.48     76554
weighted avg       0.86      0.92      0.88     76554



#### Creating a object of GradientBoostingClassifier

In [31]:
gdc_model=GradientBoostingClassifier(random_state=23)

In [32]:
gdc_model.fit(xtrain,ytrain)

GradientBoostingClassifier(random_state=23)

In [33]:
#predicting with seen Data
p3_ytrain=gdc_model.predict(xtrain)

In [34]:
accuracy_score(ytrain,p3_ytrain)

0.6363451180285467

In [35]:
#predicting with Unseen Data
p3_ytest=gdc_model.predict(xtest)

In [36]:
accuracy_score(ytest,p3_ytest)

0.6342320453536066

In [37]:
print(classification_report(ytrain,p3_ytrain),classification_report(ytest,p3_ytest))

              precision    recall  f1-score   support

           0       0.64      0.63      0.64    211093
           1       0.64      0.64      0.64    211093

    accuracy                           0.64    422186
   macro avg       0.64      0.64      0.64    422186
weighted avg       0.64      0.64      0.64    422186
               precision    recall  f1-score   support

           0       0.95      0.64      0.76     70365
           1       0.13      0.61      0.21      6189

    accuracy                           0.63     76554
   macro avg       0.54      0.62      0.49     76554
weighted avg       0.88      0.63      0.72     76554



In [61]:
gdc_model.feature_importances_

array([7.39585329e-04, 3.02695440e-03, 1.19923957e-01, 4.24585082e-02,
       2.42072398e-02, 5.38736796e-03, 1.18116391e-02, 0.00000000e+00,
       0.00000000e+00, 2.07007089e-04, 3.26202741e-03, 7.71750114e-03,
       7.01971087e-03, 8.57451433e-02, 5.96290933e-02, 2.10637048e-01,
       4.16442246e-02, 7.33670723e-02, 2.45776394e-02, 1.17008129e-03,
       5.08261947e-04, 3.80175040e-03, 0.00000000e+00, 1.03382045e-02,
       5.12016332e-02, 0.00000000e+00, 0.00000000e+00, 1.03032153e-02,
       7.09641608e-03, 0.00000000e+00, 0.00000000e+00, 1.24852365e-04,
       1.09883113e-03, 1.12022792e-01, 0.00000000e+00, 9.37794436e-04,
       8.73869853e-03, 3.07776734e-03, 2.98810739e-03, 5.07326468e-02,
       1.35201082e-04, 4.58727211e-04, 0.00000000e+00, 5.55398248e-03,
       0.00000000e+00, 6.97337537e-03, 1.37594277e-03])

In [64]:
gdc_model.learning_rate

0.1

#### Hyper Parameter tuning

In [66]:
gb_params = {'n_estimators':range(20,81,10)}

In [67]:
clf = gdc_model
gb_grid = GridSearchCV(clf,gb_params, cv=3, scoring='f1',n_jobs=-1)
gb_grid.fit(xtrain,ytrain)

NameError: name 'bg_grid' is not defined

In [68]:
print(gb_grid.best_params_)
print("Accuracy:"+ str(gb_grid.best_score_))

{'n_estimators': 80}
Accuracy:0.6344119299316665


In [70]:
gbp_ytest=gb_grid.predict(xtest)

In [71]:
gbp_ytrain=gb_grid.predict(xtrain)

In [72]:
print(classification_report(ytrain,gbp_ytrain),classification_report(ytest,gbp_ytest))

              precision    recall  f1-score   support

           0       0.63      0.63      0.63    211093
           1       0.63      0.64      0.63    211093

    accuracy                           0.63    422186
   macro avg       0.63      0.63      0.63    422186
weighted avg       0.63      0.63      0.63    422186
               precision    recall  f1-score   support

           0       0.95      0.63      0.76     70365
           1       0.13      0.61      0.21      6189

    accuracy                           0.63     76554
   macro avg       0.54      0.62      0.49     76554
weighted avg       0.88      0.63      0.72     76554



In [ ]:
print(accuracy(ytrain,gbp_ytrain))

gb_params1 ={'max_depth':range(9,16,2), 'min_samples_split':range(600,1001,200)}

clf = gdc_model
gb_grid1 = GridSearchCV(clf,gb_params1, cv=3, scoring='f1',n_jobs=-1)
gb_grid1.fit(xtrain,ytrain)
print(gb_grid1.best_params_)
print("Accuracy:"+ str(gb_grid1.best_score_))

gbp1_ytest=gb_grid1.predict(xtest)

gbp1_ytrain=gb_grid1.predict(xtrain)

print(classification_report(ytrain,gbp1_ytrain),classification_report(ytest,gbp1_ytest))

#### AdaBoostClassifier Alg

In [39]:
abc_model=AdaBoostClassifier(base_estimator=lg_model,random_state=23)

In [40]:
abc_model.fit(xtrain,ytrain)

AdaBoostClassifier(base_estimator=LogisticRegression(random_state=23),
                   random_state=23)

In [41]:
abc_ytrain=abc_model.predict(xtrain)

In [42]:
p4_ytrain=abc_model.predict_proba(xtrain)[:,1]
p4_ytrain

array([0.498543  , 0.49751103, 0.49363478, ..., 0.50412047, 0.49885866,
       0.50276684])

In [46]:
p4_ytrain1=[]
for i in p4_ytrain:
    if(i>.4996):
        p4_ytrain1.append(1)
    else:
        p4_ytrain1.append(0)
p4_ytrain1

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,


In [43]:
abc_ytest=abc_model.predict(xtest)

In [44]:
p4_ytest=abc_model.predict_proba(xtest)[:,1]
p4_ytest

array([0.49349791, 0.49621772, 0.49683894, ..., 0.49916654, 0.49552855,
       0.50035763])

In [ ]:
p4_ytest1=[]
for i in p4_ytest:
    if(i>.52):
        p4_ytest1.append(1)
    else:
        p4_ytest1.append(0)
len(p4_ytest1),len(ytest)

In [47]:
print(classification_report(ytest,p4_ytest1))

              precision    recall  f1-score   support

           0       0.95      0.59      0.73     70365
           1       0.12      0.63      0.20      6189

    accuracy                           0.60     76554
   macro avg       0.53      0.61      0.47     76554
weighted avg       0.88      0.60      0.69     76554



In [48]:
print(classification_report(ytrain,abc_ytrain),classification_report(ytest,abc_ytest))

              precision    recall  f1-score   support

           0       0.62      0.61      0.61    211093
           1       0.62      0.62      0.62    211093

    accuracy                           0.62    422186
   macro avg       0.62      0.62      0.62    422186
weighted avg       0.62      0.62      0.62    422186
               precision    recall  f1-score   support

           0       0.95      0.61      0.74     70365
           1       0.12      0.61      0.20      6189

    accuracy                           0.61     76554
   macro avg       0.53      0.61      0.47     76554
weighted avg       0.88      0.61      0.70     76554



In [32]:
#ensembling 
#Test_df=pd.DataFrame({'ytest':ytest,'p1_ytest':p1_ytest,'p2_ytest':p2_ytest,'p3_ytest':p3_ytest})
#Test_df

In [33]:
#Test_df['pro_ytest']=Test_df[['p1_ytest','p2_ytest','p3_ytest']].sum(axis=1)/len(['p1_ytest','p2_ytest','p3_ytest'])

In [ ]:
#pro_ytest=np.array(Test_df['pro_ytest'].astype('int64'))
#pro_ytest

In [ ]:
#accuracy_score(ytest,pro_ytest)

In [ ]:
#from mlxtend.feature_selection import SequentialFeatureSelector as sfs

In [ ]:
# Backward elimination
#sfs1 = sfs(lg_model, k_features=56, forward=False, verbose=1, scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
#sfs1=sfs1.fit(X,Y)

In [ ]:
#Mod_fea=list(sfs1.k_feature_names_)
#Mod_fea

In [ ]:
#Forward Selection
#sfs2 = sfs(lg_model,k_features=55, forward=True, verbose=1, scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
#sfs2=sfs2.fit(X,Y)